In [0]:
from __future__ import absolute_import, division, print_function

!pip install -q tensorflow-gpu==2.0.0-alpha0
import tensorflow_datasets as tfds
import tensorflow as tf

In [0]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()

In [0]:
##bert embedding size
bert_embedding_len = 768

In [57]:
##build tensor flow rnn model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(bert_embedding_len, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

W0330 16:36:46.661199 140000310613888 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f5388bf0630>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0330 16:36:46.670457 140000310613888 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f5388bf06d8>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


In [58]:
#Mount drive files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [59]:
import io
import os
import pandas as pd

#get biopaper data 
trainCsvPath = 'drive/My Drive/Year5/csc4006 Final Project/data/abstract_id_BertEnc_label.csv'
df = pd.read_csv(trainCsvPath,sep='\t' )
df.head()

,paperId,label,encoding
0,505487,neuroscience,"[0.2794487476348877, -0.01726824790239334, 0.0..."
1,508549,cancer biology,"[0.6084952354431152, 0.2251240611076355, -0.23..."
2,508200,cell biology,"[0.7863617539405823, 0.07360804826021194, -0.1..."
3,508341,bioengineering,"[0.5454180240631104, 0.16974613070487976, -0.2..."
4,508176,molecular biology,"[0.7510332465171814, -0.008052060380578041, -0..."


In [0]:
df = df[pd.notnull(df['label'])]

In [61]:
df.shape

(27568, 3)

In [0]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [0]:
import ast
def string_to_list(string):
  return ast.literal_eval(string)
  

In [0]:
df['encoding'] = df['encoding'].apply(lambda x: ast.literal_eval(x))


In [65]:
df.label.unique().shape

(27,)

In [0]:
from sklearn.preprocessing import LabelEncoder
encoding_len = len(df.encoding[0])
encoding_len

label = df.label

le = LabelEncoder()
outputs = le.fit_transform(label)
outputs = outputs.reshape(-1,1)

In [0]:
unq_tokens = set()

In [68]:
for enc in df.encoding:
  unq_tokens.update(enc)
print(len(unq_tokens))

18002755


In [0]:
max_words = len(unq_tokens)

In [70]:
768*df.shape[0]

21172224

In [71]:
#split training and testing data (70:30)
from sklearn.model_selection import train_test_split
trainDf, testDf = train_test_split(df,stratify=df['label'], test_size = 0.3, random_state = 11)

trainDf.shape,testDf.shape

((19297, 3), (8271, 3))

In [0]:
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

In [93]:
df.shape

(27568, 3)

In [0]:

sentence_index = [0]


embedding_matrix = 0

##each abstract is treated as one embedding 
MAX_SEQUENCE_LENGTH = 1

##len of bert embedding
EMBEDDING_DIM = 768

embedding_layer = tf.keras.layers.Embedding(27568 + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [92]:
model = tf.keras.Sequential([
    embedding_layer,
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

W0330 17:24:03.397824 140000310613888 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f538e5392e8>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


AttributeError: ignored